In [1]:
from pynq import Overlay
from pynq.lib.video import *
import time

overlay = Overlay("/home/xilinx/base2.bit")

In [2]:
imgen_vdma = overlay.video.axi_vdma_0.readchannel
videoMode = common.VideoMode(640, 480, 24)
imgen_vdma.mode = videoMode
imgen_vdma.start()

hdmi_out = overlay.video.hdmi_out
hdmi_out._vdma = overlay.video.axi_vdma #Use the correct VDMA!
hdmi_out.configure(videoMode)
hdmi_out.start()

pixgen = overlay.pixel_generator_0

In [10]:
import socket
import pickle
import numpy as np

# Initialize TCP client
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 9999))
print("Connected to server.")

done = False
pause = 0

try:
    while True:
        # Receive data from server
        data = b""
        packet = client_socket.recv(99999999)

        # Deserialize received data
        message = pickle.loads(packet)
        if not done and isinstance(message, np.array):
            done=True
            flag=False
            for i in range(1080):
                pixgen.register_map.gp1 = flag
                pixgen.register_map.gp0 = message[i]
                flag=not flag
            pixgen.register_map.gp2 = done
        elif isinstance(message, str):
            pixgen.register_map.gp2 = 1
            print("paused")
            message=""
        else:
            pixgen.register_map.gp2 = 0
        frame = imgen_vdma.readframe()
        hdmi_out.writeframe(frame)
            
except Exception as e:
    print(f"Error: {e}")

finally:
    client_socket.close()
    print("Connection closed.")

59.606307133226146
